In [ ]:
from datetime import date, timedelta
import calendar
import csv
from tkinter import filedialog
from tkinter import Tk

class LoanAmortization:
    def __init__(self, loan_amount, interest_rate, start_date, loan_duration, payment_frequency, grace_period=0, additional_payments=None, additional_payment_dates=None, amortized=True):
        self.loan_amount = loan_amount
        self.interest_rate = interest_rate / 100 / payment_frequency  # Convert to periodic rate
        self.start_date = start_date
        self.loan_duration = loan_duration * payment_frequency  # Convert to number of payments
        self.payment_frequency = payment_frequency
        self.grace_period = grace_period
        self.additional_payments = additional_payments or []
        self.additional_payment_dates = [int(date) for date in additional_payment_dates] if additional_payment_dates else []
        self.amortized = amortized

        self.balance = self.loan_amount
        self.payment = self.calculate_payment()
        self.schedule = self.generate_schedule()

    def calculate_payment(self):
        if self.amortized:
            r = self.interest_rate
            n = self.loan_duration
            p = self.loan_amount

            payment = p * (r * (1 + r) ** n) / ((1 + r) ** n - 1)
        else:
            payment = self.loan_amount * self.interest_rate
        return payment

    def generate_schedule(self):
        schedule = []
        date_obj = self.start_date
        balance = self.loan_amount
        payment = self.payment

        for period in range(1, self.loan_duration + 1):
            interest = balance * self.interest_rate

            if self.amortized:
                if period <= self.grace_period:
                    principal = 0
                    payment = interest
                else:
                    principal = payment - interest
                    principal = min(principal, balance)
                    balance -= principal
            else:
                principal = 0
                if period == self.loan_duration:
                    principal = balance
                    payment = principal  # Set payment to be equal to principal for the final payment
                    interest = 0  # Set interest to 0 for the final payment
                    balance = 0

            if self.additional_payments and period in self.additional_payment_dates:
                additional_payment = self.additional_payments[self.additional_payment_dates.index(period)]
                balance -= additional_payment
                principal += additional_payment

            schedule.append({
                'date': date_obj,
                'payment': payment,
                'interest': interest,
                'principal': principal,
                'balance': balance
            })

            divider = 12 // self.payment_frequency
            date_obj = self.add_months(date_obj, divider)

        # If there is a grace period and the loan is amortized, recalculate the schedule for the remaining periods
        if self.grace_period > 0 and self.amortized:
            remaining_balance = schedule[self.grace_period]['balance']
            remaining_periods = self.loan_duration - self.grace_period
            if remaining_periods > 0:
                start_date_after_grace = self.add_months(self.start_date, self.grace_period * divider)
                new_schedule = self.generate_amortization_schedule(remaining_balance, self.interest_rate, start_date_after_grace, remaining_periods, self.payment_frequency)
                schedule = schedule[:self.grace_period + 1] + new_schedule

        return schedule

    def generate_amortization_schedule(self, remaining_balance, interest_rate, start_date, remaining_periods, payment_frequency):
        schedule = []
        date_obj = start_date
        balance = remaining_balance
        r = interest_rate
        n = remaining_periods
        p = remaining_balance

        payment = p * (r * (1 + r) ** n) / ((1 + r) ** n - 1)

        for period in range(1, remaining_periods + 1):
            interest = balance * r
            principal = payment - interest
            principal = min(principal, balance)
            balance -= principal

            schedule.append({
                'date': date_obj,
                'payment': payment,
                'interest': interest,
                'principal': principal,
                'balance': balance
            })

            divider = 12 // payment_frequency
            date_obj = self.add_months(date_obj, divider)

        return schedule

    def add_months(self, date_obj, months):
        month = date_obj.month
        year = date_obj.year
        month += months

        while month > 12:
            month -= 12
            year += 1
        last_day_of_month = calendar.monthrange(year, month)[1]
        day = min(date_obj.day, last_day_of_month)
        return date(year, month, day)

    def print_schedule(self):
        for entry in self.schedule:
            print(f"{entry['date']} | {entry['payment']:.2f} | {entry['interest']:.2f} | {entry['principal']:.2f} | {entry['balance']:.2f}")
    
 
def blend_schedules(schedules):
    blended_schedule = []
    payment_dates = sorted({entry['date'] for schedule in schedules for entry in schedule})

    for payment_date in payment_dates:
        total_payment = total_interest = total_principal = total_balance = 0

        for schedule in schedules:
            entry = next((entry for entry in schedule if entry['date'] == payment_date), None)
            if entry:
                total_payment += entry['payment']
                total_interest += entry['interest']
                total_principal += entry['principal']
                total_balance += entry['balance']

        blended_schedule.append({
            'date': payment_date,
            'payment': total_payment,
            'interest': total_interest,
            'principal': total_principal,
            'balance': total_balance
        })

    return blended_schedule

def write_to_csv(schedule, filename):
    with open(filename, 'w', newline='') as csvfile:
        fieldnames = ['date', 'payment', 'interest', 'principal', 'balance']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()

        for entry in schedule:
            writer.writerow({
                'date': entry['date'],
                'payment': entry['payment'],
                'interest': entry['interest'],
                'principal': entry['principal'],
                'balance': entry['balance']
            })

def read_loans_from_csv(filename):
    loans = []
    with open(filename, 'r', encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile, skipinitialspace=True)
        for row in reader:
            if any(row.values()):  # Check if the row contains any value
                loan_amount = float(row['loan_amount'])
                interest_rate = float(row['interest_rate'])
                start_date = date(int(row['start_year']), int(row['start_month']), int(row['start_day']))
                loan_duration = int(row['loan_duration'])
                payment_frequency = int(row['payment_frequency'])
                grace_period = int(row['grace_period'])
                additional_payments = []
                additional_payment_dates = []
                amortized=int(row['amortized'])

                loan = LoanAmortization(loan_amount, interest_rate, start_date, loan_duration, payment_frequency, grace_period, additional_payments, additional_payment_dates, amortized)
                loans.append(loan)

    return loans

def open_file_dialog():
    root = Tk()
    root.withdraw()  # Hide the Tkinter root window
    file_path = filedialog.askopenfilename(title="Select CSV file", filetypes=[("CSV files", "*.csv")])
    return file_path

# Prompt the user to upload the CSV file
csv_filename = open_file_dialog()

if csv_filename:
    loans = read_loans_from_csv(csv_filename)

    schedules = [loan.schedule for loan in loans]
    blended_schedule = blend_schedules(schedules)

    print("Blended Schedule:")
    for entry in blended_schedule:
        print(f"{entry['date']} | {entry['payment']:.2f} | {entry['interest']:.2f} | {entry['principal']:.2f} | {entry['balance']:.2f}")

    write_to_csv(blended_schedule, 'blended_schedule.csv')
else:
    print("No file selected.")